In [0]:
import pyspark.pandas as ps

In [0]:
path = 'dbfs:/FileStore/dados_transformados/data.parquet'
df_data = ps.read_parquet(path)

In [0]:
df_data.info()

In [0]:
df_data = df_data.dropna()

In [0]:
df_data.info()

In [0]:
df_data['artists_song'] = df_data.artists + ' - ' + df_data.name

In [0]:
df_data.head(5)

In [0]:
df_data.info()

In [0]:
X = df_data.columns.to_list()
X.remove('artists')
X.remove('id')
X.remove('name')
X.remove('artists_song')
X.remove('release_date')
X

In [0]:
df_data = df_data.to_spark()

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
dados_encoded_vector = VectorAssembler(inputCols=X, outputCol='features').transform(df_data)

In [0]:
dados_encoded_vector.select('features').show(truncate=False, n=5)

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')
model_scaler = scaler.fit(dados_encoded_vector)
dados_musicas_scaler = model_scaler.transform(dados_encoded_vector)

In [0]:
dados_musicas_scaler.select('features_scaled').show(truncate=False, n=5)

In [0]:
k = len(X)
k

In [0]:
from pyspark.ml.feature import PCA

In [0]:
pca = PCA(k=k, inputCol='features_scaled', outputCol='pca_features')
model_pca = pca.fit(dados_musicas_scaler)
dados_musicas_pca = model_pca.transform(dados_musicas_scaler)

In [0]:
model_pca.explainedVariance

In [0]:
sum(model_pca.explainedVariance) * 100

In [0]:
lista_valores = [sum(model_pca.explainedVariance[0:i+1]) for i in range(k)]
lista_valores

In [0]:
import numpy as np

In [0]:
k = sum(np.array(lista_valores) <= 0.7)
k

In [0]:
pca = PCA(k=k, inputCol='features_scaled', outputCol='pca_features')
model_pca = pca.fit(dados_musicas_scaler)
dados_musicas_pca_final = model_pca.transform(dados_musicas_scaler)

In [0]:
dados_musicas_pca_final.select('pca_features').show(truncate=False, n=5)

In [0]:
from pyspark.ml import Pipeline

In [0]:
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=X, outputCol='features'),
                                StandardScaler(inputCol='features', outputCol='features_scaled'),
                                PCA(k=6, inputCol='features_scaled', outputCol='pca_features')])

In [0]:
model_pca_pipeline = pca_pipeline.fit(df_data)

In [0]:
projection = model_pca_pipeline.transform(df_data)

In [0]:
projection.select('pca_features').show(truncate=False, n=5)

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
SEED = 1224

In [0]:
kmeans = KMeans(k=50, featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)

In [0]:
modelo_kmeans = kmeans.fit(projection)

In [0]:
projetion_kmeans = modelo_kmeans.transform(projection) 

In [0]:
projetion_kmeans.select(['pca_features','cluster_pca']).show()

In [0]:
from pyspark.ml.functions import vector_to_array

In [0]:
projetion_kmeans = projetion_kmeans.withColumn('x', vector_to_array('pca_features')[0])\
                                   .withColumn('y', vector_to_array('pca_features')[1])

In [0]:
projetion_kmeans.select(['x', 'y', 'cluster_pca', 'artists_song']).show()

In [0]:
import plotly.express as px

In [0]:
fig = px.scatter(projetion_kmeans.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['artists_song'])
fig.show()